In [1]:
%load_ext autoreload
%autoreload 2

In [16]:
import os

In [19]:
project = "beneaththewater"
if os.path.isdir(project):
  !rm -rf {project}
!git clone https://github.com/hzxsunshine/beneaththewater.git

Cloning into 'beneaththewater'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 50 (delta 18), reused 41 (delta 13), pack-reused 0
Receiving objects: 100% (50/50), 10.70 KiB | 10.70 MiB/s, done.
Resolving deltas: 100% (18/18), done.


In [6]:
import beneaththewater

In [7]:
from beneaththewater.utils.data_preparation import get_uieb_data, setup_colab_env
get_uieb_data()
setup_colab_env()
!find . -type f -name '._*' -delete

Downloading...
From: https://drive.google.com/uc?id=1Z5LKgmZmFRCQ6kZzHWTcV8gzeLcxHtXE
To: /content/uieb_dataset.tar.gz
100%|██████████| 1.52G/1.52G [00:17<00:00, 87.7MB/s]


In [31]:
from beneaththewater.configs.constants import path_to_data
from beneaththewater.utils.data_preparation import prepare_the_dataframe, prepare_challenge_dataframe
uieb_df = prepare_the_dataframe(path_to_data)
uieb_challenging_df = prepare_challenge_dataframe(path_to_data)

Number of files in the first directory:  890
Number of files in the second directory:  890
Number of common files:  890
Example of common files:  ['356_img_.png', '315_img_.png', '715_img_.png', '563.png', '467_img_.png']
Number of files in the directory:  60
Example of files:  ['15735.png', '3051.png', '10226.png', '3279.png', '12753.png']


In [8]:
import torch
import clip
from PIL import Image

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

100%|███████████████████████████████████████| 338M/338M [00:03<00:00, 96.5MiB/s]


In [14]:
from beneaththewater.configs.constants import select_contrastive_pairs
pairs = select_contrastive_pairs(indices=None)
print(f"Current Pairs are: ", pairs)

Current Pairs are:  [['Accurate Color Representation', 'Blue/Green Color Cast']]


In [34]:
temp_raw = uieb_df.iloc[0]["raw"]

In [35]:
image = preprocess(Image.open(temp_raw)).unsqueeze(0).to(device)
text = clip.tokenize(pairs[0]).to(device)

with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)

    logits_per_image, logits_per_text = model(image, text)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()

print("Label probs:", probs)  # prints: [[0.9927937  0.00421068 0.00299572]]

Label probs: [[0.12003732 0.87996274]]
